In [4]:
import math
import json
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

cross_fold_validations = StratifiedKFold(n_splits=10, shuffle=True)

In [3]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

class Model:
    def __init__(self,name,ml_model):
        self.name = name
        self.model = ml_model
        self.f1_scores = []
        self.roc_auc_scores = []
        self.precision_scores = []
        self.recall_scores = []
        
    def add_f1(self,x):
        self.f1_scores.append(x)
        
    def add_roc(self,x):
        self.roc_auc_scores.append(x)
        
    def add_precision(self,x):
        self.precision_scores.append(x)
        
    def add_recall(self,x):
        self.recall_scores.append(x)
        
    def summary(self):
        f1_mean = np.mean(self.f1_scores)
        f1_std = np.std(self.f1_scores)
        roc_mean = np.mean(self.roc_auc_scores)
        roc_std = np.std(self.roc_auc_scores)
        prec_mean = np.mean(self.precision_scores)
        prec_std = np.std(self.precision_scores)
        rec_mean = np.mean(self.recall_scores)
        rec_std = np.std(self.recall_scores)
        return {self.name : {
            'f1_mean' : f1_mean,
            'f1_std' : f1_std,
            'roc_mean' : roc_mean,
            'roc_std' : roc_std,
            'precision_mean' : prec_mean,
            'precision_std' : prec_std,
            'recall_mean' : rec_mean,
            'recall_std' : rec_std
        }}

def model_runner(model_list,data):
    for train, test in cross_fold_validations.split(data,labels):
        x_train = data[train]
        x_test = data[test]
        y_train = labels[train]
        y_test = labels[test]

        for m in model_list:
            m.model.fit(x_train,y_train)
            predictions = m.model.predict(x_test)
            roc_auc = roc_auc_score(y_test, predictions)
            f1 = f1_score(y_test,predictions)
            precision = precision_score(y_test, predictions, zero_division=0)
            recall = recall_score(y_test, predictions)
            m.add_f1(f1)
            m.add_roc(roc_auc)
            m.add_precision(precision)
            m.add_recall(recall)

In [8]:
# Ensemble learner

from sklearn.ensemble import VotingClassifier

models = list()
models.append(('LDA3', LinearDiscriminantAnalysis(solver='svd', priors=[0.75,0.25])))
models.append(('QDA9', QuadraticDiscriminantAnalysis(priors=[0.9,0.1])))
models.append(('LOGREG4', LogisticRegression(multi_class='ovr', max_iter=2000, penalty='l1', class_weight='balanced', solver='liblinear')))

# define the hard voting ensemble
ensemble = VotingClassifier(estimators=models, voting='hard')



In [ ]:
# UI data

